In [1]:
!pip install nnfs

In [2]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(self.inputs, self.weights ) + self.biases
        return self.output

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)
        self.dinputs = np.dot(dvalues, self.weights.T)

In [3]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        exponantial = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        normalize = exponantial / np.sum(exponantial, axis=1, keepdims=True)
        self.output = normalize
        return self.output

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)

        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)


In [4]:
class Optimizer_SGD:
    def __init__(self, learning_rate =1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
                
            weight_updates = self.momentums * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = bias_updates

        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1

In [5]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
    
        if len(y_true.shape) == 1:
            y_true = np.eye(dvalues.shape[1])[y_true]
    
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


In [6]:
class Activation_Softmax_Loss_CategoricalCrossentropy(): 
 
    def __init__(self): 
        self.activation = Activation_Softmax() 
        self.loss = Loss_CategoricalCrossentropy() 
  
    def forward(self, inputs, y_true): 
        self.activation.forward(inputs) 
        self.output = self.activation.output 
        return self.loss.calculate(self.output, y_true) 
 
    def backward(self, dvalues, y_true): 
        samples = len(dvalues) 
 
        if len(y_true.shape) == 2: 
            y_true = np.argmax(y_true, axis=1) 
 
        self.dinputs = dvalues.copy() 
        self.dinputs[range(samples), y_true] -= 1 
        self.dinputs = self.dinputs / samples

In [7]:
import numpy as np 
import nnfs 
from nnfs.datasets import spiral_data 
 
nnfs.init() 

X, y = spiral_data(samples=100, classes=3) 

dense1 = Layer_Dense(2, 3) 
activation1 = Activation_ReLU() 
dense2 = Layer_Dense(3, 3) 
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy() 

 
dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.output, y)



print(loss_activation.output[:5]) 

print('loss:', loss) 

predictions = np.argmax(loss_activation.output, axis=1) 
if  len(y.shape) ==  2: 
   y =  np.argmax(y, axis=1) 
accuracy = np.mean(predictions == y) 

print('acc:', accuracy) 

loss_activation.backward(loss_activation.output, y) 
dense2.backward(loss_activation.dinputs) 
activation1.backward(dense2.dinputs) 
dense1.backward(activation1.dinputs) 

print(dense1.dweights) 
print(dense1.dbiases) 
print(dense2.dweights) 
print(dense2.dbiases)

[[0.33333334 0.33333334 0.33333334]
 [0.33331734 0.33331832 0.33336434]
 [0.3332888  0.33329153 0.33341965]
 [0.33325943 0.33326396 0.33347666]
 [0.33323312 0.33323926 0.33352762]]
loss: 1.0984449
acc: 0.34
[[ 0.0014546   0.00082671  0.00045028]
 [ 0.00174065  0.00012617 -0.00035851]]
[[-0.00334526  0.00105437 -0.00096934]]
[[ 0.00048936  0.00102524 -0.0015146 ]
 [-0.00044266 -0.00074882  0.00119148]
 [-0.00054513  0.00084585 -0.00030072]]
[[-0.00106752 -0.00094733  0.0020148 ]]


In [8]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = Optimizer_SGD(learning_rate=0.05, decay=5e-7)

# Train in loop
for epoch in range(10001):

    # Perform a forward pass of our training data through this layer
    dense1.forward(X)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)

    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.347, loss: 1.098, lr: 0.05
epoch: 100, acc: 0.387, loss: 1.088, lr: 0.04999752512250644
epoch: 200, acc: 0.397, loss: 1.082, lr: 0.04999502549496326
epoch: 300, acc: 0.390, loss: 1.077, lr: 0.049992526117345455
epoch: 400, acc: 0.387, loss: 1.073, lr: 0.04999002698961558
epoch: 500, acc: 0.383, loss: 1.070, lr: 0.049987528111736124
epoch: 600, acc: 0.377, loss: 1.068, lr: 0.049985029483669646
epoch: 700, acc: 0.387, loss: 1.067, lr: 0.049982531105378675
epoch: 800, acc: 0.393, loss: 1.065, lr: 0.04998003297682575
epoch: 900, acc: 0.393, loss: 1.064, lr: 0.049977535097973466
epoch: 1000, acc: 0.400, loss: 1.063, lr: 0.049975037468784345
epoch: 1100, acc: 0.400, loss: 1.063, lr: 0.049972540089220974
epoch: 1200, acc: 0.407, loss: 1.062, lr: 0.04997004295924593
epoch: 1300, acc: 0.413, loss: 1.062, lr: 0.04996754607882181
epoch: 1400, acc: 0.420, loss: 1.061, lr: 0.049965049447911185
epoch: 1500, acc: 0.420, loss: 1.061, lr: 0.04996255306647668
epoch: 1600, acc: 0.420, lo